# Backtest

In [17]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class SimpleMovingAverageCross(bt.Strategy):
    params = (
        ("fast", 10),
        ("slow", 30),
    )

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)

    def next(self):
        if self.fast_sma[0] > self.slow_sma[0] and not self.position:
            self.buy()
        elif self.fast_sma[0] < self.slow_sma[0] and self.position:
            self.sell()

class CustomStrategy(bt.Strategy):
    params = (
        ('rsi_period', 14),
        ('macd_short', 12),
        ('macd_long', 26),
        ('macd_signal', 9),
        ('bbands_period', 20),
        ('bbands_devfactor', 2),
    )

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_short, period_me2=self.params.macd_long, period_signal=self.params.macd_signal)
        self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.bbands_period, devfactor=self.params.bbands_devfactor)

    def next(self):
        # Buy conditions
        if (
            self.rsi < 30 and
            self.macd.macd > self.macd.signal and
            self.data.close < self.bbands.lines.bot
        ):
            self.buy()

        # Sell conditions
        elif (
            self.rsi > 70 and
            self.macd.macd < self.macd.signal and
            self.data.close > self.bbands.lines.top
        ):
            self.sell()

# Initialize the Cerebro engine
cerebro = bt.Cerebro()

# Add the custom strategy to the Cerebro engine
cerebro.addstrategy(SimpleMovingAverageCross)

# Create a data feed from the dataset
data = CustomPandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

# Add the data feed to the Cerebro engine
cerebro.adddata(data)

# Set the initial cash balance
cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

result = results[0]

# Analyze the performance
num_trades = len(result._tradespending)
num_wins = sum(1 for trade in result._tradespending if trade.pnl > 0)
num_losses = sum(1 for trade in result._tradespending if trade.pnl < 0)
winning_rate = num_wins / num_trades * 100 if num_trades > 0 else 0

# Print the performance metrics
print('Starting Portfolio Value: %.2f' % cerebro.broker.startingcash)
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Number of Trades: %d' % num_trades)
print('Winning Trades: %d' % num_wins)
print('Losing Trades: %d' % num_losses)
print('Winning Rate: %.2f%%' % winning_rate)

# Plot the result
#cerebro.plot()

Starting Portfolio Value: 10000.00
Starting Portfolio Value: 10000.00
Final Portfolio Value: nan
Number of Trades: 0
Winning Trades: 0
Losing Trades: 0
Winning Rate: 0.00%


In [16]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2022-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class MovingAverageStrategy(bt.Strategy):
    params = (('fast', 10), ('slow', 50), ('stop_loss_percent', 0.15))

    def __init__(self):
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.order = None
        self.stop_loss = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            print('Order Submitted/Accepted:', order.ref)
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                print('BUY EXECUTED:', order.executed.price)
                self.stop_loss = order.executed.price * (1.0 - self.params.stop_loss_percent)
                print('Stop-loss price set to:', self.stop_loss)
            elif order.issell():
                print('SELL EXECUTED:', order.executed.price)
                self.stop_loss = None
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected:', order.ref)
        self.order = None

    def next(self):
        # Print moving averages, close prices, and crossover values for monitoring
        print('Fast MA:', self.fast_ma[0])
        print('Slow MA:', self.slow_ma[0])
        print('Close price:', self.data.close[0])
        print('Crossover:', self.crossover[0])

        if self.order:
            return  # Wait for the pending order to complete
        
        if not self.position:
            if self.crossover > 0:
                print('Signal to BUY')
                self.order = self.buy()
        elif self.position:
            if self.crossover < 0:
                print('Signal to SELL (Crossover)')
                self.order = self.close()
            elif self.data.close[0] <= self.stop_loss:
                print('Signal to SELL (Stop-loss hit)')
                self.order = self.close()

    def stop(self):
        print('Final Portfolio Value: %.2f' % self.broker.getvalue())

# Initialize the cerebro engine
cerebro = bt.Cerebro()

# Set the data
data = CustomPandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

# Add the data to cerebro
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(MovingAverageStrategy)

# Set the starting cash
cerebro.broker.setcash(100000.0)

# Set the commission
cerebro.broker.setcommission(commission=0.001)

# Add observers for trade and portfolio performance metrics
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.BuySell)

# Print the starting portfolio value
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
cerebro.run()

# Print the final portfolio value
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the results
#cerebro.plot()

Starting Portfolio Value: 100000.00
Fast MA: 162.32568999999998
Slow MA: 165.134464
Close price: 162.6547
Crossover: 0.0
Fast MA: 161.56629
Slow MA: 164.83826000000002
Close price: 156.7008
Crossover: 0.0
Fast MA: 160.85957
Slow MA: 164.62117600000002
Close price: 157.0089
Crossover: 0.0
Fast MA: 160.07234
Slow MA: 164.465718
Close price: 158.5197
Crossover: 0.0
Fast MA: 159.95207
Slow MA: 164.49951399999998
Close price: 165.0204
Crossover: 0.0
Fast MA: 158.99289
Slow MA: 164.433912
Close price: 155.8261
Crossover: 0.0
Fast MA: 158.56758
Slow MA: 164.329964
Close price: 156.5627
Crossover: 0.0
Fast MA: 157.5143
Slow MA: 164.08014599999998
Close price: 151.3665
Crossover: 0.0
Fast MA: 157.30924
Slow MA: 163.87373599999998
Close price: 153.8053
Crossover: 0.0
Fast MA: 156.32969
Slow MA: 163.546026
Close price: 145.8318
Crossover: 0.0
Fast MA: 154.2552
Slow MA: 163.07308
Close price: 141.9098
Crossover: 0.0
Fast MA: 153.22903
Slow MA: 162.69727999999998
Close price: 146.4391
Crossover: 0.

In [20]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])
dataset[dataset['ticker'] == 'AAPL']

,ticker,date,adjusted_close,volume
0,AAPL,2005-02-15,1.3438,2312217600
1,AAPL,2005-02-16,1.3699,1639243200
2,AAPL,2005-02-17,1.3346,1518473600
3,AAPL,2005-02-18,1.3194,1163254400
4,AAPL,2005-02-22,1.2963,1219293600
...,...,...,...,...
4527,AAPL,2023-02-09,150.6400,56007100
4528,AAPL,2023-02-10,151.0100,57409100
4529,AAPL,2023-02-13,153.8500,62199000
4530,AAPL,2023-02-14,153.2000,61707600


In [21]:
print(dataset[(dataset['ticker'] == 'AAPL') & (dataset['date'] == '2022-07-19')])

     ticker       date  adjusted_close    volume
4385   AAPL 2022-07-19        150.3113  82982400


In [14]:
print("Missing values in 'adjusted_close':", dataset['adjusted_close'].isnull().sum())

Missing values in 'adjusted_close': 0


In [15]:
min_required_data_points = max(MovingAverageStrategy.params['fast'], MovingAverageStrategy.params['slow'])
if len(dataset) < min_required_data_points:
    print("Insufficient data points for moving average calculation.")

TypeError: 'type' object is not subscriptable

# Unit Tests

In [22]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', 'adjusted_close'),
        ('high', 'adjusted_close'),
        ('low', 'adjusted_close'),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class MovingAverageStrategy(bt.Strategy):
    params = (('fast', 10), ('slow', 50), ('stop_loss_percent', 0.15), ('ticker', 'AAPL'))

    def __init__(self):
        # Shift the close prices by 1 day to calculate the moving averages of previous day's close prices
        shifted_close = self.data.close(-1)
        self.fast_ma = bt.indicators.SimpleMovingAverage(shifted_close, period=self.params.fast)
        self.slow_ma = bt.indicators.SimpleMovingAverage(shifted_close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.order = None
        self.stop_loss = None

        # Validate stop-loss percentage
        if self.params.stop_loss_percent < 0 or self.params.stop_loss_percent > 1:
            self.log('Invalid stop-loss percentage, setting to default value of 0.15')
            self.params.stop_loss_percent = 0.15

    def log(self, txt, dt=None):
        ''' Logging function for this strategy '''
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print('%s, %s' % (dt.date(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            self.log('Order Submitted/Accepted: %s' % order.ref)
            return

        if order.status == order.Completed:
            if order.isbuy():
                self.log('BUY EXECUTED: %s' % order.executed.price)
                self.stop_loss = order.executed.price * (1.0 - self.params.stop_loss_percent)
                self.log('Stop-loss price set to: %s' % self.stop_loss)
            elif order.issell():
                self.log('SELL EXECUTED: %s' % order.executed.price)
                self.stop_loss = None
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected: %s' % order.ref)
        self.order = None
    def next(self):
        # Log moving averages, close prices, and crossover values for monitoring
        self.log('Fast MA: %s' % self.fast_ma[0])
        self.log('Slow MA: %s' % self.slow_ma[0])
        self.log('Close price: %s' % self.data.close[0])
        self.log('Crossover: %s' % self.crossover[0])
        self.log('Portfolio Value: %s' % self.broker.getvalue())

        if self.order:
            return  # Wait for the pending order to complete
        
        if not self.position:
            if self.crossover > 0:
                self.log('Signal to BUY')
                self.order = self.buy()
        elif self.position:
            if self.crossover < 0:
                self.log('Signal to SELL (Crossover)')
                self.order = self.close()
            elif self.data.close[0] <= self.stop_loss:
                self.log('Signal to SELL (Stop-loss hit)')
                self.order = self.close()

    def stop(self):
        self.log('Final Portfolio Value: %.2f' % self.broker.getvalue())

#Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')
dataset = dataset.loc[idx[:, '2022-02-15':'2023-02-15'], ['adjusted_close', 'volume']]
dataset.index.names = ['ticker', 'date']
dataset = dataset.reset_index()
dataset['date'] = pd.to_datetime(dataset['date'])

#Initialize the cerebro engine
cerebro = bt.Cerebro()

#Set the data
selected_ticker = 'AAPL' # Change this to select a different stock
data = CustomPandasData(dataname=dataset[dataset['ticker'] == selected_ticker])

#Add the data to cerebro
cerebro.adddata(data)

#Add the strategy
cerebro.addstrategy(MovingAverageStrategy, ticker=selected_ticker)

#Set the starting cash
cerebro.broker.setcash(100000.0)

#Set the commission
cerebro.broker.setcommission(commission=0.001)

#Add observers for trade and portfolio performance metrics
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.BuySell)

#Print the starting portfolio value
cerebro.run()

#Plot the results
#cerebro.plot()

2022-04-29, Fast MA: 162.32568999999998
2022-04-29, Slow MA: 165.134464
2022-04-29, Close price: 156.7008
2022-04-29, Crossover: 0.0
2022-04-29, Portfolio Value: 100000.0
2022-05-02, Fast MA: 161.56629
2022-05-02, Slow MA: 164.83826000000002
2022-05-02, Close price: 157.0089
2022-05-02, Crossover: 0.0
2022-05-02, Portfolio Value: 100000.0
2022-05-03, Fast MA: 160.85957
2022-05-03, Slow MA: 164.62117600000002
2022-05-03, Close price: 158.5197
2022-05-03, Crossover: 0.0
2022-05-03, Portfolio Value: 100000.0
2022-05-04, Fast MA: 160.07234
2022-05-04, Slow MA: 164.465718
2022-05-04, Close price: 165.0204
2022-05-04, Crossover: 0.0
2022-05-04, Portfolio Value: 100000.0
2022-05-05, Fast MA: 159.95207
2022-05-05, Slow MA: 164.49951399999998
2022-05-05, Close price: 155.8261
2022-05-05, Crossover: 0.0
2022-05-05, Portfolio Value: 100000.0
2022-05-06, Fast MA: 158.99289
2022-05-06, Slow MA: 164.433912
2022-05-06, Close price: 156.5627
2022-05-06, Crossover: 0.0
2022-05-06, Portfolio Value: 1000

In [ ]:
## COmbined TEST

In [34]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2021-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', 'adjusted_close'),
        ('high', 'adjusted_close'),
        ('low', 'adjusted_close'),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class CompositeStrategy(bt.Strategy):
    params = (
        ('fast', 10),
        ('slow', 30),
        ('rsi_period', 14),
        ('macd_short', 12),
        ('macd_long', 26),
        ('macd_signal', 9),
        ('bbands_period', 20),
        ('bbands_devfactor', 2),
        ('stop_loss_percent', 0.15),
        ('ticker', 'AAPL')
    )

    def __init__(self):
        # Calculate RSI, MACD, and BBands
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_short, period_me2=self.params.macd_long, period_signal=self.params.macd_signal)
        self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.bbands_period, devfactor=self.params.bbands_devfactor)
        
        # Calculate Moving Averages for stop-loss mechanism
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.order = None
        self.stop_loss = None

        # Validate stop-loss percentage
        if self.params.stop_loss_percent < 0 or self.params.stop_loss_percent > 1:
            self.log('Invalid stop-loss percentage, setting to default value of 0.15')
            self.params.stop_loss_percent = 0.15

    def log(self, txt, dt=None):
        ''' Logging function for this strategy '''
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print('%s, %s' % (dt.date(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            self.log('Order Submitted/Accepted: %s' % order.ref)
            return

        if order.status == order.Completed:
            if order.isbuy():
                self.log('BUY EXECUTED: %s' % order.executed.price)
                self.stop_loss = order.executed.price * (1.0 - self.params.stop_loss_percent)
                self.log('Stop-loss price set to: %s' % self.stop_loss)
            elif order.issell():
                self.log('SELL EXECUTED: %s' % order.executed.price)
                self.stop_loss = None
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected: %s' % order.ref)
        self.order = None

    def next(self):
        # Generate individual signals for each indicator
        rsi_buy_signal = self.rsi < 30
        rsi_sell_signal = self.rsi > 70
        macd_buy_signal = self.macd.macd > self.macd.signal
        macd_sell_signal = self.macd.macd < self.macd.signal
        bbands_buy_signal = self.data.close < self.bbands.lines.bot
        bbands_sell_signal = self.data.close > self.bbands.lines.top

        # Log the individual signals
        self.log('RSI Buy Signal: {}'.format(rsi_buy_signal))
        self.log('RSI Sell Signal: {}'.format(rsi_sell_signal))
        self.log('MACD Buy Signal: {}'.format(macd_buy_signal))
        self.log('MACD Sell Signal: {}'.format(macd_sell_signal))
        self.log('BBands Buy Signal: {}'.format(bbands_buy_signal))
        self.log('BBands Sell Signal: {}'.format(bbands_sell_signal))

        if self.order:
            return  # Wait for the pending order to complete
        
        if not self.position:
            if rsi_buy_signal or macd_buy_signal or bbands_buy_signal:  # Buy if any buy signal
                self.log('Signal to BUY')
                self.order = self.buy()
        elif self.position:
            if rsi_sell_signal or macd_sell_signal or bbands_sell_signal:  # Sell if any sell signal
                self.log('Signal to SELL (Crossover)')
                self.order = self.close()
            elif self.data.close[0] <= self.stop_loss:
                self.log('Signal to SELL (Stop-loss hit)')
                self.order = self.close()

    def stop(self):
        self.log('Final Portfolio Value: %.2f' % self.broker.getvalue())

# Initialize the Cerebro engine
cerebro = bt.Cerebro()

# Add the composite strategy to the Cerebro engine
cerebro.addstrategy(CompositeStrategy, ticker='AAPL')

# Create a data feed from the dataset
data = CustomPandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

# Add the data feed to the Cerebro engine
cerebro.adddata(data)

# Set the initial cash balance
cerebro.broker.setcash(10000.0)

# Set the commission
cerebro.broker.setcommission(commission=0.001)

# Add observers for trade and portfolio performance metrics
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions_analyzer')
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.BuySell)

# Print the starting portfolio value
#print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest and retrieve the results and analyzers
results = cerebro.run()
result = results[0]
transactions_analyzer = result.analyzers.transactions_analyzer

# Analyze the performance
transactions_by_date = transactions_analyzer.get_analysis()
transactions = [t for trans_list in transactions_by_date.values() for t in trans_list]
num_trades = len(transactions)
# num_wins = sum(1 for t in transactions if t.price * t.size > 0)
# num_losses = sum(1 for t in transactions if t.price * t.size < 0)
# winning_rate = num_wins / num_trades * 100 if num_trades > 0 else 0

# Print the performance metrics
print('Starting Portfolio Value: %.2f' % cerebro.broker.startingcash)
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Number of Trades: %d' % num_trades)
#print('Winning Trades: %d' % num_wins)
#print('Losing Trades: %d' % num_losses)
#print('Winning Rate: %.2f%%' % winning_rate)

2021-04-05, RSI Buy Signal: False
2021-04-05, RSI Sell Signal: False
2021-04-05, MACD Buy Signal: True
2021-04-05, MACD Sell Signal: False
2021-04-05, BBands Buy Signal: False
2021-04-05, BBands Sell Signal: False
2021-04-05, Signal to BUY
2021-04-06, Order Submitted/Accepted: 13819
2021-04-06, Order Submitted/Accepted: 13819
2021-04-06, BUY EXECUTED: 124.7089
2021-04-06, Stop-loss price set to: 106.002565
2021-04-06, RSI Buy Signal: False
2021-04-06, RSI Sell Signal: False
2021-04-06, MACD Buy Signal: True
2021-04-06, MACD Sell Signal: False
2021-04-06, BBands Buy Signal: False
2021-04-06, BBands Sell Signal: False
2021-04-07, RSI Buy Signal: False
2021-04-07, RSI Sell Signal: False
2021-04-07, MACD Buy Signal: True
2021-04-07, MACD Sell Signal: False
2021-04-07, BBands Buy Signal: False
2021-04-07, BBands Sell Signal: True
2021-04-07, Signal to SELL (Crossover)
2021-04-08, Order Submitted/Accepted: 13820
2021-04-08, Order Submitted/Accepted: 13820
2021-04-08, SELL EXECUTED: 128.8095
